# CAPM Catalog and Content Summary Review

This notebook fetches and displays the catalog descriptions and content summaries for documents sourced from 'internal_capm'.

In [ ]:
import sys
import os
import pandas as pd
import psycopg2

# Add project root to path to import db_config
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    from iris.src.initial_setup.db_config import connect_to_db, LOCAL_DB_PARAMS
except ImportError as e:
    print(f"Error importing db_config: {e}")
    print("Ensure the notebook is run from the 'notebooks' directory or adjust the path.")
    LOCAL_DB_PARAMS = None # Set to None to indicate failure
    connect_to_db = None

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 500)

In [ ]:
conn = None
catalog_df = pd.DataFrame()
content_summary_df = pd.DataFrame()

if connect_to_db:
    try:
        print(f"Connecting to database: {LOCAL_DB_PARAMS['dbname']} on {LOCAL_DB_PARAMS['host']}:{LOCAL_DB_PARAMS['port']} as {LOCAL_DB_PARAMS['user']}")
        conn = connect_to_db(env='local') # Assuming local environment for notebook
        
        if conn:
            print("Connection successful.")
            
            # Fetch Catalog Data
            print("Fetching CAPM catalog data...")
            catalog_query = """
                SELECT id, document_name, document_description 
                FROM apg_catalog 
                WHERE document_source = 'internal_capm' 
                ORDER BY document_name;
            """
            catalog_df = pd.read_sql(catalog_query, conn)
            print(f"Fetched {len(catalog_df)} catalog entries.")

            # Fetch Content Summaries
            print("Fetching CAPM content summaries...")
            content_summary_query = """
                SELECT document_name, section_id, section_name, section_summary 
                FROM apg_content 
                WHERE document_source = 'internal_capm' 
                ORDER BY document_name, section_id;
            """
            content_summary_df = pd.read_sql(content_summary_query, conn)
            print(f"Fetched {len(content_summary_df)} content summary entries.")

        else:
            print("Database connection failed.")

    except (Exception, psycopg2.Error) as error:
        print(f"Error while connecting to or querying PostgreSQL: {error}")

    finally:
        # Closing database connection.
        if conn:
            conn.close()
            print("Database connection closed.")
else:
    print("Database configuration not loaded. Cannot connect to DB.")

## CAPM Catalog (`apg_catalog`)

In [ ]:
if not catalog_df.empty:
    display(catalog_df)
else:
    print("No catalog data loaded.")

## CAPM Content Summaries (`apg_content`)

In [ ]:
if not content_summary_df.empty:
    display(content_summary_df)
else:
    print("No content summary data loaded.")